In [ ]:
source('../src/requirements.R')
source('../src/utils.R')

# Read files and pre-process

In [ ]:
phase = 'lockdown' # select between 'lockdown' or 'recovery'
response_var = 'workplaces_percent_change_from_baseline_interp_ma_diff_maxmin'
filename = paste0('mobility_google_county_',phase)
pc_var_thr = '0.55' # select between '0.55' or '0.90'

In [ ]:
data.sp <- readOGR(paste0("../data/",filename,".shp"))
data.df <- read.delim(paste0("../data/",filename,'.csv'), 
                      header = TRUE, 
                      sep = ',',na.strings = "")
data.df_pc <- read.delim(paste0("../data/",filename,'_pc_scores_',pc_var_thr,'.csv'), 
                      header = TRUE, 
                      sep = ',',na.strings = "")

data.df <- merge(data.df,data.df_pc,by=c("ID","geoid"))

## Compute the density of cases per 100,000

In [ ]:
data.df['cases_cum_dens'] = data.df['cases_cum_dens']*100000

## Select the fixed effects 

In [ ]:
cov = c(colnames(select(data.df,contains("pc_"))),'cases_cum_dens')

## Compute adjecency matrix

In [ ]:
#Compute first order neighbours
data.nb <- poly2nb(data.sp, row.names = data.sp$ID)
names(data.nb) <- attr(data.nb, "region.id")

#Convert it to a binary adjacency matrix
data.adj <- nb2mat(data.nb , style = "B",zero.policy=TRUE)

## Scale response (beta likelihood)

In [ ]:
data.df[[response_var]] <- data.df[[response_var]]/100

# Models

In [ ]:
INLA:::inla.dynload.workaround() ## LINUX WORKAROUND

## Priors

In [ ]:
bym.prior <- list(
    prec = list(
        prior ="pc.prec",
        param = c(0.5/0.31, 0.01)),
    phi = list(
        prior = "pc",
        param = c(0.5, 2/3))
)

prior.iid = c(1,0.01)
prior.besag = c(1,0.001)

## Beta-model

In [ ]:
ff  <- as.formula(paste(response_var, paste(c(cov),
                                            collapse=" + "),sep='~'))

out <- inla(ff,data = data.df, 
            family = "beta", 
            control.compute = list(cpo=TRUE,waic = TRUE),
            control.predictor = list(compute = TRUE, link = 1), 
            control.fixed = list(mean = 0, prec = 1), 
            num.threads=16, 
            verbose = TRUE)

summary(out)

### Plot the marginals for the fixed parameters  

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 14)
p <- bri.fixed.plot(out, together=FALSE)
ggsave(paste0('../plots/',filename,'_',pc_var_thr,'_fixed.pdf'), width = 20, height = 14)

### Plot the marginals for the hyperparameters 

In [ ]:
options(repr.plot.width = 30, repr.plot.height = 10)
p <- bri.hyperpar.plot(out, together=FALSE)
ggsave(paste0('../plots/',filename,'_',pc_var_thr,'_hyper.pdf'), width = 30, height = 10)

In [ ]:
out_residuals <- inla_ppcheck(out, data.df[[response_var]])
out_residuals <-  out_residuals[complete.cases(out_residuals), ] 

In [ ]:
pseudoR2(out_residuals$predicted-out_residuals$observed,out_residuals$observed)

In [ ]:
options(repr.plot.width = 42, repr.plot.height = 10)
p <- ggplot_inla_ppcheck(out_residuals, CI = TRUE, binwidth=0.05)
ggsave(paste0('../plots/',filename,'_',pc_var_thr,'_residuals.pdf'), p, width = 42, height = 10)

In [ ]:
plot(p)

## Spatial Beta-model

In [ ]:
ff.sp  <- as.formula(paste(response_var, paste(c("f(ID, model = 'bym', param = c(prior.iid, prior.besag), graph=data.adj, adjust.for.con.comp = TRUE, scale.model = TRUE)",
                                                 cov
                                             ),
                                            collapse=" + "),sep='~'))

out.sp <- inla(ff.sp,data = data.df, 
            family = "beta", 
            control.compute = list(cpo=TRUE,waic = TRUE),
            control.predictor = list(compute = TRUE, link = 1), 
            control.fixed = list(mean = 0, prec = 1),
            num.threads=2,
            verbose = TRUE)
out.sp = inla.rerun(out.sp)
summary(out.sp)

data.df[,paste0(response_var, '_mean')] <- as.numeric(out.sp$summary.fitted.values[, "mean"])
data.df[,paste0(response_var, '_sd')] <- as.numeric(out.sp$summary.fitted.values[, "sd"])

### Plot the marginals for the fixed parameters  

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 14)
p <- bri.fixed.plot(out.sp, together=FALSE)
ggsave(paste0('../plots/',filename,'_',pc_var_thr,'_fixed_sp.pdf'), width = 20, height = 14)

### Plot the marginals for the hyperparameters 

In [ ]:
options(repr.plot.width = 30, repr.plot.height = 10)
p <- bri.hyperpar.plot(out.sp, together=FALSE)
ggsave(paste0('../plots/',filename,'_',pc_var_thr,'_hyper_sp.pdf'), width = 30, height = 10)

In [ ]:
out_residuals <- inla_ppcheck(out.sp, data.df[[response_var]])

In [ ]:
pseudoR2(out_residuals$predicted-out_residuals$observed,out_residuals$observed)

In [ ]:
options(repr.plot.width = 42, repr.plot.height = 10)
p <- ggplot_inla_ppcheck(out_residuals, CI = TRUE, binwidth=0.05)
ggsave(paste0('../plots/',filename,'_',pc_var_thr,'_residuals_sp.pdf'), p, width = 42, height = 10)

### Save outputs

In [ ]:
save.image(file = paste0('../data/',filename,'_',pc_var_thr,'.RData'))

write.table(data.df,paste0('../data/',filename,'_',pc_var_thr,'_output.csv'), 
            col.names = TRUE, row.names = FALSE, sep=',')

## Load model results

In [ ]:
phase = 'lockdown' # select between 'lockdown' or 'recovery'
response_var = 'workplaces_percent_change_from_baseline_interp_ma_diff_maxmin'
filename = paste0(source,'mobility_google_county_',phase)
pc_var_thr = '0.55' # select between '0.55' or '0.90'

In [ ]:
load(file = paste0("../data/",filename,'_',pc_var_thr,".RData"))

In [ ]:
inla.show.hyperspec(out.sp)

### Compute  odds percentage relative change

In [ ]:
out_fixed = round(out$summary.fixed,6)
out_fixed$sig = ifelse(0> out_fixed['0.025quant'] & 0 < out_fixed['0.975quant'], 'non_sig','sig')
out_fixed$pc_increase = c(NA,(exp(out_fixed$mean[-1])-1)*100)
out_fixed

In [ ]:
for(i in seq(1,length(out$marginals.fixed))){
    m = out$marginals.fixed[[i]]
    print(paste0(round(inla.beta.marginal.summary(m)[[1]],2)," (",round(inla.beta.marginal.summary(m)[[2]],2),")"))
}

In [ ]:
out.sp_fixed = round(out.sp$summary.fixed,6)
out.sp_fixed$sig = ifelse(0> out.sp_fixed['0.025quant'] & 0 < out.sp_fixed['0.975quant'], 'non_sig','sig')
out.sp_fixed$pc_increase = c(NA,(exp(out.sp_fixed$mean[-1])-1)*100)
out.sp_fixed

In [ ]:
for(i in seq(1,length(out.sp$marginals.fixed))){
    m = out.sp$marginals.fixed[[i]]
    print(paste0(round(inla.beta.marginal.summary(m)[[1]],2)," (",round(inla.beta.marginal.summary(m)[[2]],2),")"))
}

### Save to file posterior estimates for the spatial random effects

In [ ]:
out.sp_random = out.sp$summary.random$ID
write.table(out.sp_random, paste0('../data/',filename,'_',pc_var_thr,'_random.csv'), sep = ',', row.names = F, col.names = T)